## Exploring the reusability of the same Huffman coding tree for compressing similar data files
#### Naveen Narayanan Meyyappan and Praneeth Chandra Thota
#### Date:11/27/2019

In [41]:
import heapq
import binascii
import struct
import pickle as pkl
import os

class TreeNode:
    def __init__(self, val=0, char=''):
        self.right = None
        self.left = None
        self.val = val
        self.char = char

    def __lt__(self, other):
        return self.val < other.val


def get_frequency(file):
    frequency = {}
    with open(file) as f:
        while True:
            c = f.read(1)
            if not c:
                break
            frequency[c] = 1 if c not in frequency else frequency[c]+1

    return frequency


def build_optimal_merge_tree(file):
    frequencies = get_frequency(file)
    print("Step 1: Calculate the frequecies of each character in the file")
    print(" ")
    for x in frequencies:
        print(x,":",frequencies[x])
    heap = [TreeNode(frequencies[char], char) for char in frequencies]
    heapq.heapify(heap)
    
    while len(heap) > 1:

        left, right = heapq.heappop(heap), heapq.heappop(heap)
        parent = TreeNode(left.val+right.val)
        parent.left = left
        parent.right = right

        heapq.heappush(heap, parent)
        
    return heap[0]



def build_encode_table(root):
    
    encode_table = {}
    decode_table = {}
    stack = [["", root]]
    while stack:
        code, node = stack.pop(-1)
        if node.left:
            stack.append([code+'0', node.left])
        if node.right:
            stack.append([code+'1', node.right])
        if not node.left and not node.right:
            encode_table[node.char] = code
            decode_table[code] = node.char
    
    return encode_table, decode_table


# Just work on this function....
def encode_file(encode_table, input_file, output_file):
    
    try:
        with open(input_file, "r") as input:
            lines = input.read()
            encoded_lines = '' 
            for c in lines:
                encoded_lines += encode_table[c]
            extra_padding = 8 - len(encoded_lines) % 8
            
            encoded_lines += "0"*extra_padding
            padded_info = "{0:08b}".format(extra_padding)
            encoded_lines = padded_info + encoded_lines
            byte_data = bytearray([int(encoded_lines[i:i+8], 2) for i in range(0, len(encoded_lines), 8)])
            with open(output_file, "wb") as output:
                output.write(bytes(byte_data))
        return True

    except:
        return False



def decode_file(decode_table, input_file, output_file):

    try:
        with open(input_file, 'rb') as input:
            byte = input.read(1)
            bit_data = ''
            while byte:
                bit_data += "{0:08b}".format(ord(byte.decode('ISO 8859-1')))
                byte = input.read(1)
            padding, bit_data = bit_data[:8], bit_data[8:]
            padding = int(padding, 2)
            bit_data = bit_data[:-padding]
            with open(output_file, "w") as output:
                code = ''
                for bit in bit_data:
                    code += bit
                    if code in decode_table:
                        output.write(decode_table[code])
                        code = ''
        return True
    except:
        return False
    
if __name__ == "__main__":
    print("Exploring the reusability of the same Huffman coding tree for compressing similar data files")
    print(" ")
    print("Project By Naveen Narayanan Meyyappan and Praneeth Chandra Thota")
    print (" ")
    print("Working Directory: ", os.getcwd())
    print(" ")
    print("Let us perform the Huffman Coding on the file data.txt")
    print(" ")       
    root = build_optimal_merge_tree("data.txt")
    encode_table, decode_table = build_encode_table(root)
    print(" ")
    print("Step 2: Generate the Encoding Table for the given data set(Data.txt)")
    print(" ")
    for x in encode_table:
        print(x,":",encode_table[x])
    print(" ")
    print("Step 3: Save the encoded table as encoded.pkl for compression of other files")
    output = open('encodingtable.pkl', 'wb')
    pkl.dump(encode_table, output)
    output.close()
    output = open('decodingtable.pkl', 'wb')
    pkl.dump(decode_table, output)
    output.close()
    print(" ")
    statinfoenctable = os.stat('encodingtable.pkl')
    print("Size of the encoding and decoding tables: ", statinfoenctable.st_size)
    print(" ")
    print("Step 4: Now let us use the above encoding table to compress and decompress the sample1.txt file using the encoding table from data.txt")
    print(" ")
    if encode_file(encode_table,"sample1.txt","sample1_encoded.bin"):
        print("Compression Successful")
        print("Compressed file saved as sample1_encoded.bin")
        statinfocomdata1org = os.stat('sample1.txt')
        print("Size of the sample1.txt file is: ", statinfocomdata1org.st_size)
        statinfocomdata1 = os.stat('sample1_encoded.bin')
        print("Size of the compressed file is: ", statinfocomdata1.st_size)
        print(" ")
    else:
        print("Compression Failed")
    if decode_file(decode_table,"sample1_encoded.bin","sample1_decoded.txt"):
        print("Decompression Successful")
        print("Decompressed file saved as sample_decoded1.txt")
        print(" ")
    else:
        print("Decompression Failed")
    print("Step 5: Let us continue the above steps for sample2.txt - sample10.txt ")
    print(" ")
    if encode_file(encode_table,"sample2.txt","sample2_encoded.bin"):
        print("Compression Successful")
        print("Compressed file saved as sample2_encoded.bin")
        statinfocomdata2org = os.stat('sample2.txt')
        print("Size of the sample2.txt file is: ", statinfocomdata2org.st_size)
        statinfocomdata2 = os.stat('sample2_encoded.bin')
        print("Size of the compressed file is: ", statinfocomdata2.st_size)
        print(" ")
    if encode_file(encode_table,"sample3.txt","sample3_encoded.bin"):
        print("Compression Successful")
        statinfocomdata3org = os.stat('sample3.txt')
        print("Size of the sample3.txt file is: ", statinfocomdata3org.st_size)
        print("Compressed file saved as sample3_encoded.bin")
        statinfocomdata3 = os.stat('sample3_encoded.bin')
        print("Size of the compressed file is: ", statinfocomdata3.st_size)
        print(" ")
    if encode_file(encode_table,"sample4.txt","sample4_encoded.bin"):
        print("Compression Successful")
        statinfocomdata4org = os.stat('sample4.txt')
        print("Size of the sample4.txt file is: ", statinfocomdata4org.st_size)
        print("Compressed file saved as sample4_encoded.bin")
        statinfocomdata4 = os.stat('sample4_encoded.bin')
        print("Size of the compressed file is: ", statinfocomdata4.st_size)
        print(" ")
    if encode_file(encode_table,"sample5.txt","sample5_encoded.bin"):
        print("Compression Successful")
        statinfocomdata5org = os.stat('sample5.txt')
        print("Size of the sample5.txt file is: ", statinfocomdata5org.st_size)
        print("Compressed file saved as sample5_encoded.bin")
        statinfocomdata5 = os.stat('sample5_encoded.bin')
        print("Size of the compressed file is: ", statinfocomdata5.st_size)
        print(" ")
    if encode_file(encode_table,"sample6.txt","sample6_encoded.bin"):
        print("Compression Successful")
        statinfocomdata6org = os.stat('sample6.txt')
        print("Size of the sample6.txt file is: ", statinfocomdata6org.st_size)
        print("Compressed file saved as sample6_encoded.bin")
        statinfocomdata6 = os.stat('sample6_encoded.bin')
        print("Size of the compressed file is: ", statinfocomdata6.st_size)
        print(" ")
    if encode_file(encode_table,"sample7.txt","sample7_encoded.bin"):
        print("Compression Successful")
        statinfocomdata7org = os.stat('sample7.txt')
        print("Size of the sample7.txt file is: ", statinfocomdata7org.st_size)
        print("Compressed file saved as sample7_encoded.bin")
        statinfocomdata7 = os.stat('sample7_encoded.bin')
        print("Size of the compressed file is: ", statinfocomdata7.st_size)
        print(" ")
    if encode_file(encode_table,"sample8.txt","sample8_encoded.bin"):
        print("Compression Successful")
        statinfocomdata8org = os.stat('sample8.txt')
        print("Size of the sample8.txt file is: ", statinfocomdata8org.st_size)
        print("Compressed file saved as sample8_encoded.bin")
        statinfocomdata8 = os.stat('sample8_encoded.bin')
        print("Size of the compressed file is: ", statinfocomdata8.st_size)
        print(" ")
    if encode_file(encode_table,"sample9.txt","sample9_encoded.bin"):
        print("Compression Successful")
        statinfocomdata9org = os.stat('sample9.txt')
        print("Size of the sample9.txt file is: ", statinfocomdata9org.st_size)
        print("Compressed file saved as sample9_encoded.bin")
        statinfocomdata9 = os.stat('sample9_encoded.bin')
        print("Size of the compressed file is: ", statinfocomdata9.st_size)
        print(" ")
    if encode_file(encode_table,"sample10.txt","sample10_encoded.bin"):
        print("Compression Successful")
        statinfocomdata10org = os.stat('sample10.txt')
        print("Size of the sample10.txt file is: ", statinfocomdata10org.st_size)
        print("Compressed file saved as sample10_encoded.bin")
        statinfocomdata10 = os.stat('sample10_encoded.bin')
        print("Size of the compressed file is: ", statinfocomdata10.st_size)
        print(" ")
    print("Step 6: Now let us try to generate the compressed file of sample1.txt using the encoding table of sample1.txt")
    print("Size of the compressed file using sample1.txt encoding table is 3720 ")
    print(" ")
    print("From the above we see that the two compression ratios are:")
    print("0.539 using the conventional method ")
    print("0.654 using the new method")
    print("Eventhough the compression ratio is increased we have a faster compression algorithm")
    print("The algorithm gets faster and faster compared to the original method only when the number of files to be compressed are more than 10(each with size more than 2000)")
    print("On compressing all the 10 sample files we observe that the total size of compressed files including the header is 38627 bytes using the original Huffman compression algorithm and 34012 bytes using the new approach")

Exploring the reusability of the same Huffman coding tree for compressing similar data files
 
Project By Naveen Narayanan Meyyappan and Praneeth Chandra Thota
 
Working Directory:  C:\Users\USER\huffman
 
Let us perform the Huffman Coding on the file data.txt
 
Step 1: Calculate the frequecies of each character in the file
 
T : 5098
h : 208917
e : 854020
  : 16907
P : 6580
r : 550333
o : 559470
j : 11586
c : 327956
t : 510271
G : 3944
u : 288722
n : 561207
b : 140498
g : 184958
E : 4035
x : 22733
f : 91546
M : 7013
y : 154027
W : 2319
d : 263585
I : 2699
a : 657510

 : 837993
l : 441084
w : 57675
s : 549909
i : 688121
v : 74612
, : 422
k : 63264
! : 36
m : 225825
p : 242427
. : 2357
D : 4490
* : 48
F : 3058
V : 1636
R : 3719
B : 6092
H : 4143
C : 8456
: : 36
N : 3038
S : 8586
A : 8311
q : 13678
# : 10
5 : 242
6 : 231
@ : 7
U : 1272
O : 2716
; : 11
/ : 79
0 : 630
9 : 261
8 : 282
7 : 177
4 : 264
3 : 310
2 : 398
1 : 400
J : 1734
z : 31776
$ : 2
+ : 2
- : 47587
% : 4
' : 4461
[ : 17
= : 